<a href="https://colab.research.google.com/github/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/TA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Datos

In [1]:
import tarfile
import numpy as np
import pandas as pd

## Recopilación

In [2]:
!wget -P /content/Data https://github.com/renato145/3D-ORGAN/blob/master/datasets/arq_dataset.tar.gz?raw=true

--2022-09-28 06:34:46--  https://github.com/renato145/3D-ORGAN/blob/master/datasets/arq_dataset.tar.gz?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/renato145/3D-ORGAN/raw/master/datasets/arq_dataset.tar.gz [following]
--2022-09-28 06:34:46--  https://github.com/renato145/3D-ORGAN/raw/master/datasets/arq_dataset.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/renato145/3D-ORGAN/master/datasets/arq_dataset.tar.gz [following]
--2022-09-28 06:34:47--  https://raw.githubusercontent.com/renato145/3D-ORGAN/master/datasets/arq_dataset.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199

In [3]:
folder = "/content/Data/"
file_tar = folder + "arq_dataset.tar.gz?raw=true"
file_name = "custom_arq_dataset.npy"

In [4]:
file = tarfile.open(file_tar)
file.extract(file_name, folder)
file.close()

In [5]:
data = np.load(folder+file_name, allow_pickle=True)
data = data.reshape(1)

In [6]:
#data

## A dataframe

In [7]:
train = data[0]["train"] #'labels' 'data'
train = [train["labels"],train["data"]]
#train
#train[0][-1], train[1][0]

In [8]:
df = pd.DataFrame(train)
df = df.T
df.columns = ['labels','data']
df

,labels,data
0,bathtub,"[[[False, False, False, False, False, False, F..."
1,bathtub,"[[[False, False, False, False, False, False, F..."
2,bathtub,"[[[False, False, False, False, False, False, F..."
3,bathtub,"[[[False, False, False, False, False, False, F..."
4,bathtub,"[[[False, False, False, False, False, False, F..."
...,...,...
4918,arq,"[[[False, False, False, False, False, False, F..."
4919,arq,"[[[False, False, False, False, False, False, F..."
4920,arq,"[[[False, False, False, False, False, False, F..."
4921,arq,"[[[False, False, False, False, False, False, F..."


## Selección de tipo de objeto

In [9]:
tipos = df['labels'].unique()
tipos

array(['bathtub', 'dresser', 'chair', 'night_stand', 'toilet', 'monitor',
       'table', 'bed', 'sofa', 'desk', 'arq'], dtype=object)

In [10]:
tipo = "sofa"  #@param ['bathtub', 'dresser', 'chair', 'night_stand', 'toilet', 'monitor', 'table', 'bed', 'sofa', 'desk', 'arq']
data_tipo = df[ df['labels']==tipo ]
data_tipo

,labels,data
3111,sofa,"[[[False, False, False, False, False, False, F..."
3112,sofa,"[[[False, False, False, False, False, False, F..."
3113,sofa,"[[[False, False, False, False, False, False, F..."
3114,sofa,"[[[False, False, False, False, False, False, F..."
3115,sofa,"[[[False, False, False, False, False, False, F..."
...,...,...
3786,sofa,"[[[False, False, False, False, False, False, F..."
3787,sofa,"[[[False, False, False, False, False, False, F..."
3788,sofa,"[[[False, False, False, False, False, False, F..."
3789,sofa,"[[[False, False, False, False, False, False, F..."


# Conversión de Binvox a STL

In [15]:
!pip install voxelfuse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63.0 MB 10 kB/s 
     |████████████████████████████████| 995 kB 53.1 MB/s 
     |████████████████████████████████| 265 kB 69.2 MB/s 
     |████████████████████████████████| 8.4 MB 1.3 MB/s 
     |████████████████████████████████| 15.1 MB 34.0 MB/s 
     |████████████████████████████████| 167 kB 65.0 MB/s 
     |████████████████████████████████| 129 kB 47.2 MB/s 
     |████████████████████████████████| 235 kB 56.9 MB/s 
     |████████████████████████████████| 59.9 MB 123 kB/s 
     |████████████████████████████████| 344 kB 38.8 MB/s 
     |████████████████████████████████| 1.6 MB 68.8 MB/s 
     |████████████████████████████████| 51 kB 6.1 MB/s 
  Created wheel for py-vox-io: filename=py_vox_io-0.1-py3-none-any.whl size=5719 sha256=cf461a5a8edf385e035f1730cdccd4048f1bab5348636405d27613d227680037
  Stored in directory: /root/.cache/pip/wheels/65/81/5c

In [19]:
"""
IMPORTANTE
Para el correcto funcionamiento de los import, es necesario eliminar 
la linea: "from quad_mesh_simplify import simplify_mesh" del 
archivo mesh.py.
"""
from voxelfuse.voxel_model import VoxelModel
from voxelfuse.primitives import generateMaterials
from voxelfuse.mesh import Mesh

In [21]:
def VoxToSTL(obj):
  if __name__=='__main__':
    model = VoxelModel(obj, generateMaterials(4))  #4 is aluminium.
    mesh = Mesh.fromVoxelModel(model)
    mesh.export('mesh.stl')

In [22]:
#Caso de Prueba 
VoxToSTL(df['data'][0])

Meshing: 100%|██████████| 1742/1742 [00:00<00:00, 4529.54it/s]
